# Sample Purchase Signals based on Technical Indicators
Author: **Peeyush Sharma**; Feedback: **PSharma3@gmail.com**

This notebook takes the work captured under 'Reference_Plot_Technical_Indicators' notebook forward and lists the stocks that meet the buying criteria listed there. The buying criteria is strictly based on technical indicators. A few of these indicators are aggregated in a single selection criteria and then samples are retrieved for certain durations (bi-weekly, monthly, quarterly etc...). There is no macro analysis here whether at economy, sector, or even longer term growth perspective level for individual company. There is no guarantee that the daily market data or the derived economic indicators are correct. This notebook is for educational and demonstrative purposes only.

In [10]:
import os
import os.path
from datetime import datetime, timedelta

import pandas as pd

pd.options.mode.chained_assignment = None 
import warnings
from sqlalchemy import create_engine, text
from pandas.errors import EmptyDataError
warnings.simplefilter(action='ignore', category=FutureWarning)

In [11]:
BASE_DIR = '../../../../workspace/HelloPython/HistoricalMarketData/TechnicalIndicators'
DURATIONS = (14, 30, 90, 200) # Roughly for bi-weekly, monthly, quarterly, and 200 days moving averages and other tech indicators

In [12]:
try:
    DB = os.environ["DB"]
    DB_USER = os.environ["DB_USER"]
    DB_PWD = os.environ["DB_PWD"]
except KeyError:
    raise Exception("Required environment variables DB_USER and DB_PWD not set")
DB_URL = 'mysql+mysqlconnector://' + DB_USER + ':' + DB_PWD + '@localhost/' + DB
ENGINE = create_engine(DB_URL)

In [13]:
# Retrieve data for Technical Indicators from pre-calculated CSVs
def generate_file_path(symbol, date=None):
    if date is not None:
        str_date = datetime.strftime(date, '%Y%m%d')
        file_name = symbol.lower()+'_'+str_date+'.csv'
        file_path = os.path.join(BASE_DIR, file_name)
    else: 
        file_name = symbol.lower()+'.csv'
        file_path = os.path.join(BASE_DIR, file_name)
    if file_path is None:
            print('Could not find file for symbol:{}'.format(symbol))
    # print(file_path)
    return file_path, file_name

In [14]:
def retrieve_ticker_close_data(symbol, str_date_from = None, str_date_to = None):
    if str_date_from is None:
        dt_from = datetime(1999, 12, 31)
    else: dt_from = datetime.strptime(str_date_from, '%Y-%m-%d')
    if str_date_to is None:
        dt_to = datetime.now() - timedelta(days=1)
    else: dt_to = datetime.strptime(str_date_to, '%Y-%m-%d')

    file_path, _ = generate_file_path(symbol)
    if file_path is not None:
        try:
            dfrm = pd.read_csv(file_path)
            dfrm['date'] = pd.to_datetime(dfrm['date'])
            dfrm.set_index('date', inplace=True)
            dfrm = dfrm.loc[dt_from: dt_to, :]
            dfrm = dfrm[~dfrm.index.duplicated()]
            return dfrm
        except FileNotFoundError as e:
            print('Exception reading input data for symbol {}.'.format(symbol.upper()))
            print(e)
            return None
        except EmptyDataError as e:
            print(f'No technical indicators found for {symbol.upper()}. Skipping')
    else: return None

In [15]:
def buying_opps(dfrm, duration):
    """
    Identifies dates for buying opportunities based on different critera. The criteria here
    is hard coded based on earlier experiments - this can be optimized further.
    :param dfrm:
    :param duration:
    :return:
    """
    dates_lows_for_buy_opps = [ date for date in dfrm.index if dfrm.loc[date, 'oscillator_'+str(duration)] < 25 and dfrm.loc[date, 'pcntleVolume_'+str(duration)] > 90 and dfrm.loc[date, 'pcntleStdDevs_'+str(duration)] > 90]
    return dates_lows_for_buy_opps

In [16]:
# dfrm = retrieve_ticker_close_data('NVDA')
# buying_dates = buying_opps(dfrm, duration)
# print(type(buying_dates))
# print(buying_dates)

In [17]:
if __name__ == "__main__":
    duration = 90
    symbols_w_buy_opp_dates = dict()
    last_quarter = datetime.now() - timedelta(days=30)

    query = 'select distinct symbol from industrybackground where SnP500 like 1'
    # query = 'select distinct symbol from industrybackground where SnP500 like 1 and symbol not \
    # in (select distinct symbol from equities_historic_data where date like "2019-12-20")'

    with ENGINE.connect() as conn:
        res = conn.execute(text(query))
    dfrm_lst_symbols = pd.DataFrame(res.mappings().all())
    symbols = dfrm_lst_symbols['symbol'].tolist()

    for symbol in symbols:
        dfrm = retrieve_ticker_close_data(symbol)
        if dfrm is None or dfrm.empty:
            print(f'Np data received for symbol {symbol}. Skipping...')
            continue
        buy_dates = buying_opps(dfrm, duration)
        symbols_w_buy_opp_dates[symbol] = buy_dates

    for symbol, dates in symbols_w_buy_opp_dates.items():
        dates_recent = [date.date() for date in dates if date > last_quarter]
        if len(dates_recent) > 0:
            print(f'{symbol.upper()} has had buying opp in the last month on the following dates: ({dates_recent})')

A has had buying opp in the last month on the following dates: ([datetime.date(2025, 4, 7), datetime.date(2025, 4, 8), datetime.date(2025, 4, 9), datetime.date(2025, 4, 10), datetime.date(2025, 4, 11)])
AAL has had buying opp in the last month on the following dates: ([datetime.date(2025, 4, 8), datetime.date(2025, 4, 9), datetime.date(2025, 4, 10)])
AAP has had buying opp in the last month on the following dates: ([datetime.date(2025, 4, 7), datetime.date(2025, 4, 8), datetime.date(2025, 4, 9), datetime.date(2025, 4, 10), datetime.date(2025, 4, 11)])
AAPL has had buying opp in the last month on the following dates: ([datetime.date(2025, 4, 7), datetime.date(2025, 4, 8), datetime.date(2025, 4, 10)])
ACN has had buying opp in the last month on the following dates: ([datetime.date(2025, 4, 7), datetime.date(2025, 4, 8), datetime.date(2025, 4, 9), datetime.date(2025, 4, 10), datetime.date(2025, 4, 11)])
ADI has had buying opp in the last month on the following dates: ([datetime.date(2025,